## Value Iteration vs. Policy Iteration

While both policy iteration and value iteration can be used to solve the control problem, value iteration is often cited as a more efficient technique. This is mainly due to the fact that value iteration combines policy evaluation and policy improvement into one step. More specifically, policy iteration requires that we evaluate policy until convergence before doing a single policy improvement step whereas value iteration *only* requires one iteration of policy evaluation before moving to the policy improvement phase. Furthermore, with value iteration we don't need to wait for the entire kth iteration of V to be available before we start calculating the (k+1)th iteration of V. Rather, we can just update it "in place." Since policy improvement uses **argmax**, by taking the **max**, we're just doing the next policy evaluation step without calculating policy explicity. 

In the most simplistic terms, the steps for value iteration can be summed up as follows:

1. Initialize all V0(s) = 0 for all states
2. Compute the value of V1(s) given all the values of V0(s)
2. Repeat until the values don't change i.e. convergence is reached

## Value iteration in deterministic Grid World environment

To further illustrate value iteration concepts, I'm going to again explore the code previously used in the 'gridworld problem.' Similiar to my experiements with policy iteration, I will also test different values for gamma to see what, if any change it has on determining the optimal policy.


In [3]:
# https://deeplearningcourses.com/c/artificial-intelligence-reinforcement-learning-in-python
# https://www.udemy.com/artificial-intelligence-reinforcement-learning-in-python
from __future__ import print_function, division
from builtins import range
# Note: you may need to update your version of future
# sudo pip install -U future


import numpy as np
from grid_world import standard_grid, negative_grid
from iterative_policy_evaluation import print_values, print_policy

SMALL_ENOUGH = 1e-3
GAMMA = 0.9
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')

# this is deterministic
# all p(s',r|s,a) = 1 or 0

if __name__ == '__main__':
  # this grid gives you a reward of -0.1 for every non-terminal state
  # we want to see if this will encourage finding a shorter path to the goal
  grid = standard_grid()

  # print rewards
  print("rewards:")
  print_values(grid.rewards, grid)

  # state -> action
  # we'll randomly choose an action and update as we learn
  policy = {}
  for s in grid.actions.keys():
    policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)

  # initial policy
  print("initial policy:")
  print_policy(policy, grid)

  # initialize V(s)
  V = {}
  states = grid.all_states()
  for s in states:
    # V[s] = 0
    if s in grid.actions:
      V[s] = np.random.random()
    else:
      # terminal state
      V[s] = 0

  # repeat until convergence
  # V[s] = max[a]{ sum[s',r] { p(s',r|s,a)[r + gamma*V[s']] } }
  count = 0
  while True:
    count += 1
    biggest_change = 0
    for s in states:
      old_v = V[s]

      # V(s) only has value if it's not a terminal state
      if s in policy:
        new_v = float('-inf')
        for a in ALL_POSSIBLE_ACTIONS:
          grid.set_state(s)
          r = grid.move(a)
          v = r + GAMMA * V[grid.current_state()]
          if v > new_v:
            new_v = v
        V[s] = new_v
        biggest_change = max(biggest_change, np.abs(old_v - V[s]))
        
    print(count,biggest_change)

    if biggest_change < SMALL_ENOUGH:
      break

  # find a policy that leads to optimal value function
  for s in policy.keys():
    best_a = None
    best_value = float('-inf')
    # loop through all possible actions to find the best current action
    for a in ALL_POSSIBLE_ACTIONS:
      grid.set_state(s)
      r = grid.move(a)
      v = r + GAMMA * V[grid.current_state()]
      if v > best_value:
        best_value = v
        best_a = a
    policy[s] = best_a

  # our goal here is to verify that we get the same answer as with policy iteration
  print("values:")
  print_values(V, grid)
  print("policy:")
  print_policy(policy, grid)
  print(count)

rewards:
---------------------------
 0.00| 0.00| 0.00| 1.00|
---------------------------
 0.00| 0.00| 0.00|-1.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|
initial policy:
---------------------------
  L  |  R  |  D  |     |
---------------------------
  L  |     |  U  |     |
---------------------------
  L  |  D  |  L  |  D  |
1 0.733894342227
2 0.254122076696
3 0.0737801756191
4 0.0079215805718
5 0
values:
---------------------------
 0.81| 0.90| 1.00| 0.00|
---------------------------
 0.73| 0.00| 0.90| 0.00|
---------------------------
 0.66| 0.73| 0.81| 0.73|
policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  R  |  U  |  L  |
5


In looking specifically at the value iteration loop, it looks very similar to policy evaluation except now we're looping through all the possible actions and taking the maximum value. It breaks when the biggest change is below the 'small enough' threshold. Next, we take the optimal value function and find the optimal policy (we loop through all the actions and find the action that gives us the best future reward.

Based on the output/resutls, it looks like we get the same values as the policy iteration code. It looks like values did not change after 4 iterations. 

## Changing Gamma
Next, I'm going to decrease gamma, thereby increasing the discount and imposing more importance on acheiving the immediate reward vs. future rewards. Let's see if we get the same results as with policy iteration i.e. changing gamma to .05 led to 95% decrease.

In [4]:
from __future__ import print_function, division
from builtins import range
# Note: you may need to update your version of future
# sudo pip install -U future


import numpy as np
from grid_world import standard_grid, negative_grid
from iterative_policy_evaluation import print_values, print_policy

SMALL_ENOUGH = 1e-3
GAMMA = 0.05
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')

# this is deterministic
# all p(s',r|s,a) = 1 or 0

if __name__ == '__main__':
  # this grid gives you a reward of -0.1 for every non-terminal state
  # we want to see if this will encourage finding a shorter path to the goal
  grid = standard_grid()

  # print rewards
  print("rewards:")
  print_values(grid.rewards, grid)

  # state -> action
  # we'll randomly choose an action and update as we learn
  policy = {}
  for s in grid.actions.keys():
    policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)

  # initial policy
  print("initial policy:")
  print_policy(policy, grid)

  # initialize V(s)
  V = {}
  states = grid.all_states()
  for s in states:
    # V[s] = 0
    if s in grid.actions:
      V[s] = np.random.random()
    else:
      # terminal state
      V[s] = 0

  # repeat until convergence
  # V[s] = max[a]{ sum[s',r] { p(s',r|s,a)[r + gamma*V[s']] } }
  count = 0
  while True:
    count += 1
    biggest_change = 0
    for s in states:
      old_v = V[s]

      # V(s) only has value if it's not a terminal state
      if s in policy:
        new_v = float('-inf')
        for a in ALL_POSSIBLE_ACTIONS:
          grid.set_state(s)
          r = grid.move(a)
          v = r + GAMMA * V[grid.current_state()]
          if v > new_v:
            new_v = v
        V[s] = new_v
        biggest_change = max(biggest_change, np.abs(old_v - V[s]))
        
    print(count,biggest_change)

    if biggest_change < SMALL_ENOUGH:
      break

  # find a policy that leads to optimal value function
  for s in policy.keys():
    best_a = None
    best_value = float('-inf')
    # loop through all possible actions to find the best current action
    for a in ALL_POSSIBLE_ACTIONS:
      grid.set_state(s)
      r = grid.move(a)
      v = r + GAMMA * V[grid.current_state()]
      if v > best_value:
        best_value = v
        best_a = a
    policy[s] = best_a

  # our goal here is to verify that we get the same answer as with policy iteration
  print("values:")
  print_values(V, grid)
  print("policy:")
  print_policy(policy, grid)
  print(count)

rewards:
---------------------------
 0.00| 0.00| 0.00| 1.00|
---------------------------
 0.00| 0.00| 0.00|-1.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|
initial policy:
---------------------------
  D  |  R  |  L  |     |
---------------------------
  U  |     |  L  |     |
---------------------------
  R  |  L  |  R  |  D  |
1 0.872098210339
2 0.043604910517
3 0.00216999529037
4 6.170682481e-05
values:
---------------------------
 0.00| 0.05| 1.00| 0.00|
---------------------------
 0.00| 0.00| 0.05| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|
policy:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  R  |  U  |  L  |
4


Indeed, with value iteration we also saw a 95% decrease when moving away from the terminal state.